In [1]:
using EtherArrays
using StaticArrays
using KernelAbstractions
using oneAPI

In [4]:
using ForwardDiff

In [2]:
const kBackend = oneAPI.oneAPIBackend()
const Tcontainer = oneAPI.oneArray
const Tf = Float32

Float32

In [8]:
@inline function f(x)
    T = promote_type(Tf, eltype(x))
    return MVector{2, T}(x[1] * x[2], x[1] + x[2]) # xy, x+y
end

f (generic function with 1 method)

In [5]:
data_cpu = zeros(Tf, 2, 6)
data_cpu[1, 1] = 1
data_cpu[1, 2] = 2
data_cpu[2, 1] = 3
data_cpu[2, 2] = 4
data_gpu = Tcontainer(data_cpu)

2×6 oneArray{Float32, 2, oneAPI.oneL0.DeviceBuffer}:
 1.0  2.0  0.0  0.0  0.0  0.0
 3.0  4.0  0.0  0.0  0.0  0.0

In [9]:
@kernel function test_forwarddiff!(data)
    idx = @index(Global)
    vec = E2Vector{2}(idx, 1, data)
    mat = E2SquareMatrix{2}(idx, 3, data)
    jac = ForwardDiff.jacobian(f, vec)
    mat .= jac
end

In [18]:
test_forwarddiff!(kBackend, 2)(data_gpu, ndrange=(2,))
KernelAbstractions.synchronize(kBackend)

In [19]:
data_gpu

2×6 oneArray{Float32, 2, oneAPI.oneL0.DeviceBuffer}:
 1.0  2.0  2.0  1.0  1.0  1.0
 3.0  4.0  4.0  1.0  3.0  1.0